In [26]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch
from PIL import Image
from huggingface_hub import login
import os
from google.colab import userdata

# Вход в Hugging Face с токеном из секретов
login(token=userdata.get('HF_TOKEN'))
print("✅ Успешно вошли в Hugging Face!")

# === 1. Путь к изображению ===
image_path = "/content/drive/MyDrive/AI/детекция_картинки/3.JPG"

# Проверка существования файла
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Изображение не найдено по пути: {image_path}")

# === 2. Загрузка изображения ===
try:
    image = Image.open(image_path).convert("RGB")
    print("✅ Изображение успешно загружено.")
except Exception as e:
    raise RuntimeError(f"Не удалось открыть изображение: {e}")

# === 3. Загрузка модели ===
model_id = "llava-hf/llava-1.5-7b-hf"

if 'model' not in locals():
    print("Загружаем процессор...")
    processor = AutoProcessor.from_pretrained(model_id)

    print("Загружаем модель...")
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto"
    )

# === 4. Промпт и обработка ===
prompt = "USER: <image>\nНа картинке дворцовая площадь Санкт-Петербурга. Опиши, сколько людей на этой картинке. ASSISTANT:\n"

# 🔥 КРИТИЧЕСКИ ВАЖНО: порядок аргументов — сначала текст, потом изображение
inputs = processor(
    text=prompt,
    images=image,          # ← именно объект PIL.Image, НЕ строка!
    return_tensors="pt"
).to("cuda", torch.float16)

# === 5. Генерация ===
print("Генерация ответа...")
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False
    )

response = processor.decode(output[0], skip_special_tokens=True)
print("\n🧠 Ответ модели:\n", response)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Успешно вошли в Hugging Face!
✅ Изображение успешно загружено.
Генерация ответа...

🧠 Ответ модели:
 USER:  
На картинке дворцовая площадь Санкт-Петербурга. Опиши, сколько людей на этой картинке. ASSISTANT:

На картинке много людей, которые стоят и стоят на дворцовой площади Санкт-Петербурга. Они разбросаны по всей площади, и некоторые из них могут быть вдалеке друг от друга. Всего на картинке примерно 15 человек.
